In [1]:
from methods.shap_methods import shap_file_path
from methods.pca_methods import PCAExplain
import pickle
import os

d:\Python Projects\Actuarial_NN_Forecasting\test_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# SHAP Values

In [2]:
VARIABLE = "CPIAUCSL"  # Select from CPIAUCSL, GS5, RPI and UNRATE

In [3]:
with open(shap_file_path(VARIABLE), "rb") as input_file:
    shap_object = pickle.load(input_file)

In [4]:
shap_object.plot_importance()

In [19]:
shap_object.plot_dependence("PCA_2(t-2)")

In [16]:
shap_object.plot_dependence("PCA_10(t-2)")

# PCA

In [6]:
PCAExplain.plot_pca_component(2)

In [17]:
PCAExplain.plot_pca_component(10)